In [4]:
listenToPath = '/user/s292129/data/exam_09_2020/ListenToSongs.txt'
usersPath = '/user/s292129/data/exam_09_2020/Users.txt'
songsPath = "/user/s292129/data/exam_09_2020/Songs.txt"

In [6]:
usersRDD = sc.textFile(usersPath)

# We retrieve only the lines associated with young users

youngUsersRDD =  usersRDD.filter(lambda line: int(line.split(",")[4]) > 1999)

# We map each line associated with young users in a pair (userID, None) in order to be able to perform a join operation

uid_NoneRDD  = youngUsersRDD.map(lambda line: ( line.split(",")[0], None ) )

listenToRDD = sc.textFile(listenToPath).cache()

# We retrieve the lines with songs listened to in the last two years

listenToLastTwoYearsRDD =  listenToRDD.filter( lambda line: line.split(",")[2].split("_")[0] >= "2018/09/14" and line.split(",")[2].split("_")[0] <= "2020/09/13" )

uid_sidRDD = listenToLastTwoYearsRDD.map( lambda line: ( line.split(",")[1], line.split(",")[0] ) ).cache()

# We use this join operation to retrieve the songs that have been listened by young users in the last 2 years

uid_sidNone = uid_sidRDD.join( uid_NoneRDD )

# We get the songs never listened by young users in the last 2 years subtracting the songs that have been listened by young users from all the songs that have been listened by someone in the last two years

sidNeverListenedByYoungs = uid_sidRDD.values().subtract( uid_sidNone.values().keys() )

sidNeverListenedByYoungs.collect()



# TASK 2 : For each song, the year in which it is characterized by its highest yearly popularity



def mapFunc(line):

    fields = line.split(',')

    sid = fields[0]

    year = fields[2].split("_")[0].split('/')[0]

    uid  = fields[1]

    return ( (sid, year), uid )



mappedListenToRDD =  listenToRDD.map( mapFunc )

distinctListenToRDD = mappedListenToRDD.distinct()

sidYear_one = distinctListenToRDD.mapValues(lambda v: 1)

sidYear_popularity = sidYear_one.reduceByKey(lambda v1, v2: v1+v2)

sid_yearPopularity = sidYear_popularity.map( lambda pair: ( pair[0][0], ( pair[0][1], pair[1] ) ) )

# We use this defined function to get the maximum yearly popularity and its respective year using a reduceByKey transformation 

def returnMaxAndYear( v1, v2):

    pop1 = v1[1]

    pop2 = v2[1]

    year1 = v1[0]

    year2 = v2[0]

    if pop1 > pop2 or ( pop1 == pop2 and year1 < year2):

        return (year1, pop1)

    else: 

        return (year2, pop2)



sid_maxYearAndPopularity = sid_yearPopularity.reduceByKey(returnMaxAndYear)

sid_maxYearAndPopularity.map( lambda pair: (pair[0], pair[1][0]) ).collect()

[('SID10', '2019'), ('SID11', '2016'), ('SID12', '2019')]

In [2]:
listenToRDD = sc.textFile(listenToPath).cache()
# schema: sid, uid, startTS, endTS
usersRDD = sc.textFile(usersPath)
# schema: uid, name, surname, gender, yearofbirth

In [ ]:
filteredRDD = usersRDD.filter(lambda line: int(line.split(',')[4]) >= 1999)

In [ ]:
youngUidsRDD = filteredRDD.map(lambda line: (line.split(',')[0]))

In [ ]:
uid_songStartTS_RDD = listenToRDD.map(lambda line: (line.split(',')[1],\
                                           (line.split(','[0], line.split(',')[2]))))

In [ ]:
sid_startTSRDD =  youngUidsRDD.join(uid_songStartTS_RDD).map(lambda pair: (pair[1][0],\
                                                         pair[1][1])
# schema : (sid, startTS)

In [ ]:
sid_startTSRDD.groupByKey()

def filterNeverListenedSongs(pair):
    timestamps = list(pair[1])
    timestamps2 = timestamps[1:]
    neverListened
    for ts1 in timestamps:
        for ts2 in timestamps2:
            if ts1

# Task B

In [3]:
distinctSidYear_uidRDD = listenToRDD.map(lambda line: ((line.split(',')[0],\
                                                        line.split(',')[2].split('_')[0].split('/')[0]),\
                                                        line.split(',')[1])).distinct()

In [4]:
sidYear_onesRDD = distinctSidYear_uidRDD.mapValues(lambda v: +1)

In [5]:
sidYear_numOfListenersRDD = sidYear_onesRDD.reduceByKey(lambda v1, v2: v1+v2)

In [6]:
sid_yearNumOfListenersRDD = sidYear_numOfListenersRDD.map(lambda pair: (pair[0][0], (pair[0][1], pair[1])))

In [10]:
def yearForPopularity(pair1, pair2):
    year1 = pair1[0]
    year2 = pair2[0]
    numListeners1 = pair1[1]
    numListeners2 = pair2[1]
    if (numListeners1 > numListeners2):
        return pair1
    elif (numListeners1 == numListeners2 and year1 < year2): 
        return pair1
    elif (numListeners1 == numListeners2 and year1 > year2):
        return pair2
    else:
        return pair2
        
sid_yearNOFperPopRDD = sid_yearNumOfListenersRDD.reduceByKey(yearForPopularity)

In [11]:
sid_yearNOFperPopRDD.map(lambda pair: (pair[0], pair[1][0])).collect()

[('SID10', '2019'), ('SID11', '2016'), ('SID12', '2019')]